In [27]:
import pandas as pd
from gensim.models import Word2Vec
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import torch.optim as optim
import math
from sklearn.feature_extraction.text import TfidfTransformer
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer


In [28]:
column_names = ['type','title','text']
df = pd.read_csv('./all_after_mapping.tsv',sep='\t',names=column_names)
df.shape

(35546, 3)

In [29]:
tokenlizeword = np.load('tokenlizeword0225_nopunct.npy',allow_pickle=True)


In [30]:
labels = df['type'].values
labels = np.array(labels)

In [31]:
print(type(tokenlizeword[0]))
count = 0
for k in range(tokenlizeword.shape[0]):
    tokenlizeword[k] = np.array(tokenlizeword[k])
    count+=tokenlizeword[k].shape[0]
print(count/tokenlizeword.shape[0])
print(type(tokenlizeword[0]))
li = []
for k in range(tokenlizeword.shape[0]):
    li.append(' '.join(tokenlizeword[k]))
li = np.array(li)
li.shape

<class 'list'>
264.60656613965006
<class 'numpy.ndarray'>


(35546,)

In [32]:
vectorizer = CountVectorizer(max_features=1024)
X = vectorizer.fit_transform(li)
word = vectorizer.get_feature_names()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
x = tfidf.toarray()
print(x.shape)

['10', '100', '10日', '10月', '11', '11日', '11月', '12', '12月', '13', '14', '15', '15日', '16', '17', '18', '19', '1日', '1月', '20', '2016年', '2017年', '2018年', '2019', '2019年', '2020', '2020年', '21', '22', '23', '23日', '24', '25', '26', '29日', '2日', '2月', '30', '30日', '31日', '35', '3日', '3月', '40', '4月', '50', '500', '5g', '60', '6日', '6月', '7日', '8日', '9月', 'po', '一些', '一切', '一同', '一定', '一度', '一樣', '一直', '一般', '一起', '一路', '上午', '上漲', '下午', '不僅', '不同', '不少', '不斷', '不會', '不滿', '不用', '不能', '不要', '不過', '世界', '並非', '中共', '中午', '中國', '中央', '中心', '中華', '中華民國', '主任', '主動', '主委', '主席', '主管', '主要', '主題', '之前', '之後', '之間', '乘客', '事件', '事實', '事情', '事業', '互動', '亞洲', '交易', '交流', '交通', '交通部', '人口', '人員', '人士', '人數', '人民', '人民幣', '人潮', '人物', '什麼', '今天', '今年', '今日', '今晚', '他們', '代表', '以上', '以下', '以來', '以及', '任何', '企業', '伊朗', '但是', '位於', '作品', '作業', '作為', '你們', '使用', '來到', '來自', '侯友宜', '保持', '保護', '保障', '信心', '個人', '個案', '候選人', '做出', '做好', '停止', '健康', '傳出', '傳播', '傳染', '傳統', '像是', '價值', '價格', '儘管', '先前', '先

In [33]:
print(x.shape)
dev_x = x[:5000]
dev_y = labels[:5000]

test_x = x[5000:10000]
test_y = labels[5000:10000]

train_x = x[10000:]
train_y = labels[10000:]

(35546, 1024)


In [34]:
from torch.utils.data import Dataset,DataLoader
class TFIDFdataset(Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.len = x.shape[0]
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x,y
    def __len__(self):
        return self.len

In [35]:
BATCH_SIZE = 32
trainset = TFIDFdataset(train_x,train_y)
trainloader = DataLoader(trainset,batch_size=BATCH_SIZE,drop_last=True)


In [36]:
num_classes = 7
input_dim  = x.shape[1]
class TFIDFmodel(nn.Module):
    def __init__(self):
        super(TFIDFmodel, self).__init__()
        self.fc1 = nn.Linear(input_dim,1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [41]:
model = TFIDFmodel()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print('device:',device)
model = model.float()
model = model.to(device)
model.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(20):
    running_loss = 0
    for data in trainloader:
        x,y = [t.to(device) for t in data]
        x = x.float()
        y = y.float()
        optimizer.zero_grad()
        output = model(x)
        y = y.long()
        loss = criterion(output, y)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    torch.save(model, 'model_TFIDF_1024.pkl')



device: cuda:0
Epoch: 0001 cost = 1.916712
Epoch: 0002 cost = 1.783167
Epoch: 0003 cost = 1.554767
Epoch: 0004 cost = 1.251404
Epoch: 0005 cost = 1.028769
Epoch: 0006 cost = 0.894432
Epoch: 0007 cost = 0.801158
Epoch: 0008 cost = 0.729628
Epoch: 0009 cost = 0.673833
Epoch: 0010 cost = 0.630082
Epoch: 0011 cost = 0.595777
Epoch: 0012 cost = 0.567887
Epoch: 0013 cost = 0.545101
Epoch: 0014 cost = 0.525903
Epoch: 0015 cost = 0.509389
Epoch: 0016 cost = 0.495220
Epoch: 0017 cost = 0.482773
Epoch: 0018 cost = 0.471768
Epoch: 0019 cost = 0.462150
Epoch: 0020 cost = 0.453477


In [42]:
testset = TFIDFdataset(test_x,test_y)
testloader = DataLoader(testset,batch_size=BATCH_SIZE)


In [43]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    predictions_withoutmax = None
    correct = 0
    total = 0
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                x,y = [t.to("cuda:0") for t in data if t is not None]
                x = x.float()
                y = y.float()
            outputs  = model(x)

            after_softmax = F.softmax(outputs, dim=1)
            _, pred = torch.max(after_softmax, 1)

            # 用來計算訓練集的分類準確率
            if compute_acc:
                total += y.shape[0]
                correct += (pred == y).sum().item()

            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
                
            if predictions_withoutmax is None:
                predictions_withoutmax = after_softmax
            else:
                predictions_withoutmax = torch.cat((predictions_withoutmax,after_softmax))
    
    if compute_acc:
        acc = correct / total
        return predictions , predictions_withoutmax, acc
    return predictions_withoutmax
    

In [44]:
pred , pred_probi ,acc = get_predictions(model,testloader,True)
print(acc)

0.7754
